In [1]:
# Import dependencies 
import numpy as np
from twitter import *
import pandas as pd 
import datetime

In [2]:
#The tokens and consumers keys are my API authentication to use the Twitter API.  I'm using the twitter library as a python
#wrapper to call twitter request through python.
token = '1212463191910842368-l00ryMICeXQXXawl8w2zazEoFS7xDf'
token_secret = 'hbzjFRq1r2ygkydPXfrwrhm5zTa54F3mYEm2wi7Q09DgA'
consumer_key = 'g8V7jnF3dqMfg4LRhtobYB4Pl'
consumer_secret = 'Y0SHSL0H9X9gCtuy07mJ3cp144DS2JhwX4Uvgda2ph8NvIUswJ'

t = Twitter(
        auth=OAuth(token, token_secret, consumer_key, consumer_secret))

In [3]:
type(t)

twitter.api.Twitter

In [4]:
# The function below gets a person's timeline
# Note: Count can't be greater than 200
x = t.statuses.user_timeline(screen_name="BernieSanders", count=20, include_rts=False, tweet_mode = 'extended')

In [5]:
# Creating a for loop to inserts Hillary's tweets in a list
Bernie_tweet = []
for x in x:
    Bernie_tweet.append(x)

In [8]:
Bernie_tweet[0:5]

[{'created_at': 'Tue Jan 07 22:43:48 +0000 2020',
  'id': 1214679053468393473,
  'id_str': '1214679053468393473',
  'full_text': "We must not view America only as a population of disconnected individuals.\n\nWe must view ourselves as part of “an inescapable network of mutuality, tied in a single garment of destiny,” as Dr. King put it.\n\nThat is how we'll realize our vision of justice and dignity for all.",
  'truncated': False,
  'display_text_range': [0, 275],
  'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []},
  'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'in_reply_to_screen_name': None,
  'user': {'id': 216776631,
   'id_str': '216776631',
   'name': 'Bernie Sanders',
   'screen_name': 'BernieSanders',
   'location': 'Vermont',
   'description': 'U.S. Senator from Vermont and c

In [9]:
type(Bernie_tweet)

list

In [10]:
# I just want to focus on ids and tweets
for element in Bernie_tweet:
    print (element['id'])
    print (element['full_text'])
    print('---')

1214679053468393473
We must not view America only as a population of disconnected individuals.

We must view ourselves as part of “an inescapable network of mutuality, tied in a single garment of destiny,” as Dr. King put it.

That is how we'll realize our vision of justice and dignity for all.
---
1214660857755557891
As president, I will reverse this disastrous rule on day one of my presidency. Together, we will guarantee every person with a disability the right to live in the community with dignity and independence.
---
1214660855784194048
10,000 Americans died waiting for Social Security Disability Insurance in 2017. Instead of addressing this crisis, Trump wants to make it harder for people with disabilities to get the help they need to get by.

Unacceptable. We must protect and expand SSDI and SSI. https://t.co/B0XX76AARE
---
1214652812451360769
If we get involved in a war in Iran, in all likelihood it will be even worse than the war in Iraq. https://t.co/7ns9Hnm7oo
---
1214613793

In [11]:
#(TweetMiner) I found this function that easily organizes the tweets to input them into a panda dataframe

class TweetMiner(object):

    
    def __init__(self, t, result_limit = 200):
        
        self.t = t        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user="BernieSanders", mine_retweets=False, max_pages=20):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            
            if last_tweet_id:
                statuses   =   self.t.statuses.user_timeline(screen_name=user, count=self.result_limit, max_id=last_tweet_id - 1, include_rts=mine_retweets, tweet_mode = 'extended')
                statuses = [x for x in statuses]
            else:
                statuses   =   self.t.statuses.user_timeline(screen_name=user, count=self.result_limit, include_rts=mine_retweets, tweet_mode = 'extended')
                statuses = [x for x in statuses]
                
            for item in statuses:
                # Using try except here.
                # When retweets = 0 we get an error (GetUserTimeline fails to create a key, 'retweet_count')
                try:
                    mined = {
                        'tweet_id':        item['id'],
                        'handle':          item['user']['screen_name'],
                        'retweet_count':   item['retweet_count'],
                        'text':            item['full_text'],
                        'mined_at':        datetime.datetime.now(),
                        'created_at':      item['created_at'],
                    }
                
                except:
                        mined = {
                        'tweet_id':        item['id'],
                        'handle':          item['user']['screen_name'],
                        'retweet_count':   0,
                        'text':            item['full_text'],
                        'mined_at':        datetime.datetime.now(),
                        'created_at':      item['created_at'],
                    }
                
                last_tweet_id = item['id']
                data.append(mined)
                
            page += 1
            
        return data

In [12]:
# Result limit == count parameter from our GetUserTimeline()
miner = TweetMiner(t, result_limit=200)

In [13]:
bernie = miner.mine_user_tweets(user="BernieSanders")
elizabeth = miner.mine_user_tweets(user="ewarren")

In [14]:
# each tweet is dictionary within a list
bernie[0:5]

[{'tweet_id': 1214679053468393473,
  'handle': 'BernieSanders',
  'retweet_count': 823,
  'text': "We must not view America only as a population of disconnected individuals.\n\nWe must view ourselves as part of “an inescapable network of mutuality, tied in a single garment of destiny,” as Dr. King put it.\n\nThat is how we'll realize our vision of justice and dignity for all.",
  'mined_at': datetime.datetime(2020, 1, 7, 18, 56, 0, 510589),
  'created_at': 'Tue Jan 07 22:43:48 +0000 2020'},
 {'tweet_id': 1214660857755557891,
  'handle': 'BernieSanders',
  'retweet_count': 303,
  'text': 'As president, I will reverse this disastrous rule on day one of my presidency. Together, we will guarantee every person with a disability the right to live in the community with dignity and independence.',
  'mined_at': datetime.datetime(2020, 1, 7, 18, 56, 0, 510602),
  'created_at': 'Tue Jan 07 21:31:30 +0000 2020'},
 {'tweet_id': 1214660855784194048,
  'handle': 'BernieSanders',
  'retweet_count': 1

In [15]:
for x in range(5):
    print (bernie[x]['text'])
    print('---')

We must not view America only as a population of disconnected individuals.

We must view ourselves as part of “an inescapable network of mutuality, tied in a single garment of destiny,” as Dr. King put it.

That is how we'll realize our vision of justice and dignity for all.
---
As president, I will reverse this disastrous rule on day one of my presidency. Together, we will guarantee every person with a disability the right to live in the community with dignity and independence.
---
10,000 Americans died waiting for Social Security Disability Insurance in 2017. Instead of addressing this crisis, Trump wants to make it harder for people with disabilities to get the help they need to get by.

Unacceptable. We must protect and expand SSDI and SSI. https://t.co/B0XX76AARE
---
If we get involved in a war in Iran, in all likelihood it will be even worse than the war in Iraq. https://t.co/7ns9Hnm7oo
---
We need more jobs and education. Not more jails and incarceration.
---


In [16]:
for x in range(5):
    print (elizabeth[x]['text'])
    print('--')

@Adaripp @TeamWarren Thank you for joining @TeamWarren in Las Vegas, @AdaRipp—we can’t wait to have you back again soon!
--
Our grassroots donors are pretty surprised when I call to say thanks—someone even thought I was a recording. But I promise, it's really me! And if you donate now, I might just call you tonight to say thanks: https://t.co/1wHTXr1sDs https://t.co/dX5cWYMKpJ
--
Kate McKinnon is really fabulous. I'm with @SethMeyers—big kudos to the @NBCSNL costuming department! https://t.co/slzFNaH8dX
--
@PanickedFoodie Under #MedicareForAll, mental health care will be treated as what it is—critical health care. #CripTheVote
--
We cannot ask our military to solve problems that cannot be solved militarily. We have to use every diplomatic and economic tool we have—and we need a sound strategy guiding us. Trump's reckless, dangerous actions are only escalating this crisis and putting our military at risk. https://t.co/x348lAlU6u
--


In [17]:
# Turning Bernie's tweet into a dataframe:
bernie_df = pd.DataFrame(bernie)
bernie_df.head()

,created_at,handle,mined_at,retweet_count,text,tweet_id
0,Tue Jan 07 22:43:48 +0000 2020,BernieSanders,2020-01-07 18:56:00.510589,823,We must not view America only as a population ...,1214679053468393473
1,Tue Jan 07 21:31:30 +0000 2020,BernieSanders,2020-01-07 18:56:00.510602,303,"As president, I will reverse this disastrous r...",1214660857755557891
2,Tue Jan 07 21:31:29 +0000 2020,BernieSanders,2020-01-07 18:56:00.510605,1240,"10,000 Americans died waiting for Social Secur...",1214660855784194048
3,Tue Jan 07 20:59:32 +0000 2020,BernieSanders,2020-01-07 18:56:00.510607,1140,"If we get involved in a war in Iran, in all li...",1214652812451360769
4,Tue Jan 07 18:24:29 +0000 2020,BernieSanders,2020-01-07 18:56:00.510609,4163,We need more jobs and education. Not more jail...,1214613793315205120


In [18]:
# Doing the same for Elizabeth Warren
miner = TweetMiner(t, result_limit=200)
elizabeth_tweets = miner.mine_user_tweets("ewarren", max_pages=30)

In [19]:
elizabeth_df = pd.DataFrame(elizabeth_tweets)

In [20]:
elizabeth_df.head()

,created_at,handle,mined_at,retweet_count,text,tweet_id
0,Tue Jan 07 23:22:02 +0000 2020,ewarren,2020-01-07 18:56:20.371472,4,@Adaripp @TeamWarren Thank you for joining @Te...,1214688676065755137
1,Tue Jan 07 22:19:32 +0000 2020,ewarren,2020-01-07 18:56:20.371482,124,Our grassroots donors are pretty surprised whe...,1214672944242270208
2,Tue Jan 07 21:27:03 +0000 2020,ewarren,2020-01-07 18:56:20.371484,464,Kate McKinnon is really fabulous. I'm with @Se...,1214659736513191941
3,Tue Jan 07 20:45:51 +0000 2020,ewarren,2020-01-07 18:56:20.371486,15,"@PanickedFoodie Under #MedicareForAll, mental ...",1214649367774343174
4,Tue Jan 07 20:19:44 +0000 2020,ewarren,2020-01-07 18:56:20.371488,834,We cannot ask our military to solve problems t...,1214642796679831552


In [21]:
bernie_df.shape

(2290, 6)

In [22]:
elizabeth_df.shape

(2953, 6)

In [23]:
# Joined elizabeth & Bernie Dataframe
tweets = pd.concat([elizabeth_df, bernie_df], axis=0)
tweets.shape

(5243, 6)

In [24]:
tweets.head()

,created_at,handle,mined_at,retweet_count,text,tweet_id
0,Tue Jan 07 23:22:02 +0000 2020,ewarren,2020-01-07 18:56:20.371472,4,@Adaripp @TeamWarren Thank you for joining @Te...,1214688676065755137
1,Tue Jan 07 22:19:32 +0000 2020,ewarren,2020-01-07 18:56:20.371482,124,Our grassroots donors are pretty surprised whe...,1214672944242270208
2,Tue Jan 07 21:27:03 +0000 2020,ewarren,2020-01-07 18:56:20.371484,464,Kate McKinnon is really fabulous. I'm with @Se...,1214659736513191941
3,Tue Jan 07 20:45:51 +0000 2020,ewarren,2020-01-07 18:56:20.371486,15,"@PanickedFoodie Under #MedicareForAll, mental ...",1214649367774343174
4,Tue Jan 07 20:19:44 +0000 2020,ewarren,2020-01-07 18:56:20.371488,834,We cannot ask our military to solve problems t...,1214642796679831552


In [25]:
# The library below convert a collection of text documents to a matrix of token counts
from sklearn.feature_extraction.text import TfidfVectorizer

# The library stores dictionary values as counts 
from collections import Counter

In [26]:
# Using TfidfVectorizer to find ngrams for Elizabeth's tweets
vect = TfidfVectorizer(ngram_range=(2,5), stop_words='english')
vect

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(2, 5), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [27]:
# Pulls all of Elizabeth's tweet text's into one giant string
summaries = "".join(elizabeth_df['text'])
summaries

'@Adaripp @TeamWarren Thank you for joining @TeamWarren in Las Vegas, @AdaRipp—we can’t wait to have you back again soon!Our grassroots donors are pretty surprised when I call to say thanks—someone even thought I was a recording. But I promise, it\'s really me! And if you donate now, I might just call you tonight to say thanks: https://t.co/1wHTXr1sDs https://t.co/dX5cWYMKpJKate McKinnon is really fabulous. I\'m with @SethMeyers—big kudos to the @NBCSNL costuming department! https://t.co/slzFNaH8dX@PanickedFoodie Under #MedicareForAll, mental health care will be treated as what it is—critical health care. #CripTheVoteWe cannot ask our military to solve problems that cannot be solved militarily. We have to use every diplomatic and economic tool we have—and we need a sound strategy guiding us. Trump\'s reckless, dangerous actions are only escalating this crisis and putting our military at risk. https://t.co/x348lAlU6u@cdubbs420 @jamie95126953 Programs and policies meant to deter opioid a

In [28]:
# vect.builder_analyzer handles the preprocessing, tokenization and n-grams generation
ngrams_summaries = vect.build_analyzer()(summaries)
ngrams_summaries

['adaripp teamwarren',
 'teamwarren thank',
 'thank joining',
 'joining teamwarren',
 'teamwarren las',
 'las vegas',
 'vegas adaripp',
 'adaripp wait',
 'wait soon',
 'soon grassroots',
 'grassroots donors',
 'donors pretty',
 'pretty surprised',
 'surprised say',
 'say thanks',
 'thanks thought',
 'thought recording',
 'recording promise',
 'promise really',
 'really donate',
 'donate just',
 'just tonight',
 'tonight say',
 'say thanks',
 'thanks https',
 'https 1whtxr1sds',
 '1whtxr1sds https',
 'https dx5cwymkpjkate',
 'dx5cwymkpjkate mckinnon',
 'mckinnon really',
 'really fabulous',
 'fabulous sethmeyers',
 'sethmeyers big',
 'big kudos',
 'kudos nbcsnl',
 'nbcsnl costuming',
 'costuming department',
 'department https',
 'https slzfnah8dx',
 'slzfnah8dx panickedfoodie',
 'panickedfoodie medicareforall',
 'medicareforall mental',
 'mental health',
 'health care',
 'care treated',
 'treated critical',
 'critical health',
 'health care',
 'care cripthevotewe',
 'cripthevotewe ask'

In [29]:
Counter(ngrams_summaries).most_common(20)

[('big structural', 214),
 ('structural change', 204),
 ('big structural change', 199),
 ('ve got', 180),
 ('health care', 138),
 ('got plan', 123),
 ('ve got plan', 123),
 ('climate change', 89),
 ('gun violence', 88),
 ('donald trump', 83),
 ('ll fight', 74),
 ('grassroots movement', 74),
 ('trump administration', 73),
 ('make sure', 71),
 ('student loan', 61),
 ('fight hard', 61),
 ('climate crisis', 59),
 ('loan debt', 57),
 ('plan https', 56),
 ('make big', 56)]

In [30]:
#Doing the same process for Bernie
vect = TfidfVectorizer(ngram_range=(2,5), stop_words='english')

summaries = "".join(bernie_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[('health care', 247),
 ('new deal', 103),
 ('green new', 99),
 ('green new deal', 99),
 ('donald trump', 86),
 ('fossil fuel', 85),
 ('working people', 81),
 ('white house', 76),
 ('young people', 65),
 ('health insurance', 62),
 ('student debt', 60),
 ('billionaire class', 56),
 ('american people', 56),
 ('climate change', 56),
 ('united states', 53),
 ('climate crisis', 53),
 ('wall street', 52),
 ('corporate greed', 52),
 ('working class', 51),
 ('join live', 48)]

In [17]:
ticker = ['$GOOGL', '$GOOG', '$MO', '$AMZN', '$AEE', '$AAL', '$AEP', '$AXP']

In [18]:
ticker

['$GOOGL', '$GOOG', '$MO', '$AMZN', '$AEE', '$AAL', '$AEP', '$AXP']

In [19]:
for x in ticker:
    t.search.tweets(q=f'${x}', include_rts=False, tweet_mode='extended')

TwitterHTTPError: Twitter sent status 429 for URL: 1.1/search/tweets.json using parameters: (include_rts=False&oauth_consumer_key=g8V7jnF3dqMfg4LRhtobYB4Pl&oauth_nonce=1843601419521487912&oauth_signature_method=HMAC-SHA1&oauth_timestamp=1578447645&oauth_token=1212463191910842368-l00ryMICeXQXXawl8w2zazEoFS7xDf&oauth_version=1.0&q=%24%24GOOGL&tweet_mode=extended&oauth_signature=BtUfrZdtsczzUNCtTLFS7KLkHv8%3D)
details: {'errors': [{'message': 'Rate limit exceeded', 'code': 88}]}

###### Wine search to look for #wine in the full text.  Twee_mode='extended' reflects the full tweet, not truncated.
wine = t.search.tweets(q=f'${ticker}', include_rts=False, tweet_mode='extended')

In [15]:
type(wine)

twitter.api.TwitterDictResponse

In [16]:
wine

{'statuses': [{'created_at': 'Wed Jan 08 01:30:31 +0000 2020',
   'id': 1214721007094767616,
   'id_str': '1214721007094767616',
   'full_text': 'PopSockets Announces New Product Launch: PopPower Home Charger, Powered by NuCurrent at the Consumer Electronics Show $AAPL https://t.co/u1MatpV0f3',
   'truncated': False,
   'display_text_range': [0, 147],
   'entities': {'hashtags': [],
    'symbols': [{'text': 'AAPL', 'indices': [118, 123]}],
    'user_mentions': [],
    'urls': [{'url': 'https://t.co/u1MatpV0f3',
      'expanded_url': 'https://newsfilter.io/articles/popsockets-announces-new-product-launch-poppower-home-charger-powered-by-nucurrent-at-the-consumer-e-d357990cf1028b6a231d3aaffd612c14',
      'display_url': 'newsfilter.io/articles/popso…',
      'indices': [124, 147]}]},
   'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
   'source': '<a href="https://newsfilter.io" rel="nofollow">Newsfilter.io</a>',
   'in_reply_to_status_id': None,
   'in_reply_to_status_

In [20]:
wine_list = wine['statuses']

In [21]:
type(wine_list)

list

In [22]:
wine_list

[{'created_at': 'Wed Jan 08 01:30:31 +0000 2020',
  'id': 1214721007094767616,
  'id_str': '1214721007094767616',
  'full_text': 'PopSockets Announces New Product Launch: PopPower Home Charger, Powered by NuCurrent at the Consumer Electronics Show $AAPL https://t.co/u1MatpV0f3',
  'truncated': False,
  'display_text_range': [0, 147],
  'entities': {'hashtags': [],
   'symbols': [{'text': 'AAPL', 'indices': [118, 123]}],
   'user_mentions': [],
   'urls': [{'url': 'https://t.co/u1MatpV0f3',
     'expanded_url': 'https://newsfilter.io/articles/popsockets-announces-new-product-launch-poppower-home-charger-powered-by-nucurrent-at-the-consumer-e-d357990cf1028b6a231d3aaffd612c14',
     'display_url': 'newsfilter.io/articles/popso…',
     'indices': [124, 147]}]},
  'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
  'source': '<a href="https://newsfilter.io" rel="nofollow">Newsfilter.io</a>',
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to

In [37]:
# Showing how #wine reflects within the full text
for element in wine_list[0:5]:
    print (element['id'])
    print (element['full_text'])
    print('---')

1214697293846179840
RT @visit_temecula: Come join Chapin Family Vineyards for a glass of wine with a spectacular view! Thanks for sharing this beautiful view 📷…
---
1214697283079458816
#winetasting in #manhattanbeach @PasquaWinery #carmenetwine #stefanofarinawine #lucasandlewellen @broadsidewine $10/tasting with discounts on the wines purchased from the flight #wine #familyowned #shopsmall #shoplocal #boxsigns #funnywinequotes https://t.co/B45PGDRHRt
---
1214697221561704451
New Ways Drones Are Changing Vineyard Management https://t.co/xkPBCRZh8g via @SevenFiftyDaily #wine #vineyardmanagement
---
1214697221398188033
From back in 2016 - 5 #Wine Rules That Really Matter https://t.co/8SVV4kDyPm from @WineSpectator #winelover
---
1214697010303000578
RT @CaraMiaSG: #WiningHourChat returns tonight and we’re kicking off the new year with @WineTravelerCom! Join us at 9PM ET🥂✈️ #wine #travel…
---


In [38]:
#(TweetMiner) I found this function that easily organizes the tweets to input them into a panda dataframe

class TweetMiner(object):

    
    def __init__(self, t, result_limit = 200):
        
        self.t = t        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, topic="#wine", mine_retweets=False, max_pages=20):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            
            if last_tweet_id:
                statuses   =   self.t.search.tweets(q='#wine', count=self.result_limit, max_id=last_tweet_id - 1, include_rts=mine_retweets, tweet_mode='extended')
                statuses = [x for x in statuses['statuses']]
            else:
                statuses   =   self.t.search.tweets(q='#wine', count=self.result_limit, include_rts=mine_retweets, tweet_mode='extended')
                statuses = [x for x in statuses['statuses']]
                
            for item in statuses:
                # Using try except here.
                # When retweets = 0 we get an error (GetUserTimeline fails to create a key, 'retweet_count')
                try:
                    mined = {
                        'tweet_id':        item['id'],
                        'handle':          item['user']['screen_name'],
                        'retweet_count':   item['retweet_count'],
                        'text':            item['full_text'],
                        'mined_at':        datetime.datetime.now(),
                        'created_at':      item['created_at'],
                    }
                
                except:
                        mined = {
                        'tweet_id':        item['id'],
                        'handle':          item['user']['screen_name'],
                        'retweet_count':   0,
                        'text':            item['full_text'],
                        'mined_at':        datetime.datetime.now(),
                        'created_at':      item['created_at'],
                    }
                
                last_tweet_id = item['id']
                data.append(mined)
                
            page += 1
            
        return data

In [39]:
miner = TweetMiner(t, result_limit=200)

In [40]:
wine = miner.mine_user_tweets(topic="#wine")

In [41]:
wine

[{'tweet_id': 1214697293846179840,
  'handle': 'JulieHouseOfWow',
  'retweet_count': 1,
  'text': 'RT @visit_temecula: Come join Chapin Family Vineyards for a glass of wine with a spectacular view! Thanks for sharing this beautiful view 📷…',
  'mined_at': datetime.datetime(2020, 1, 7, 18, 56, 34, 761755),
  'created_at': 'Tue Jan 07 23:56:17 +0000 2020'},
 {'tweet_id': 1214697283079458816,
  'handle': 'BacchusWineMB',
  'retweet_count': 0,
  'text': '#winetasting in #manhattanbeach @PasquaWinery #carmenetwine #stefanofarinawine #lucasandlewellen @broadsidewine $10/tasting with discounts on the wines purchased from the flight #wine #familyowned #shopsmall #shoplocal #boxsigns #funnywinequotes https://t.co/B45PGDRHRt',
  'mined_at': datetime.datetime(2020, 1, 7, 18, 56, 34, 761766),
  'created_at': 'Tue Jan 07 23:56:14 +0000 2020'},
 {'tweet_id': 1214697221561704451,
  'handle': 'GaragisteFest',
  'retweet_count': 0,
  'text': 'New Ways Drones Are Changing Vineyard Management https://t.c

In [42]:
wine_tweet_df = pd.DataFrame(wine)
wine_tweet_df.head()

,created_at,handle,mined_at,retweet_count,text,tweet_id
0,Tue Jan 07 23:56:17 +0000 2020,JulieHouseOfWow,2020-01-07 18:56:34.761755,1,RT @visit_temecula: Come join Chapin Family Vi...,1214697293846179840
1,Tue Jan 07 23:56:14 +0000 2020,BacchusWineMB,2020-01-07 18:56:34.761766,0,#winetasting in #manhattanbeach @PasquaWinery ...,1214697283079458816
2,Tue Jan 07 23:56:00 +0000 2020,GaragisteFest,2020-01-07 18:56:34.761769,0,New Ways Drones Are Changing Vineyard Manageme...,1214697221561704451
3,Tue Jan 07 23:56:00 +0000 2020,pomarjunction,2020-01-07 18:56:34.761771,0,From back in 2016 - 5 #Wine Rules That Really ...,1214697221398188033
4,Tue Jan 07 23:55:09 +0000 2020,Dracaenawines,2020-01-07 18:56:34.761773,5,RT @CaraMiaSG: #WiningHourChat returns tonight...,1214697010303000578


In [43]:
wine_list = (wine_tweet_df.text)

In [44]:
type(wine_list)

pandas.core.series.Series

In [45]:
#Vectorizing the wine df
vect = TfidfVectorizer(ngram_range=(2,5), stop_words='english')

summaries = "".join(wine_tweet_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[('monsoonvalleyuk win', 557),
 ('win bottles', 555),
 ('bottles fantastic', 555),
 ('fantastic wine', 555),
 ('wine enter', 555),
 ('enter simply', 555),
 ('simply follow', 555),
 ('follow monsoonvalleyuk', 555),
 ('monsoonvalleyuk rt', 555),
 ('rt post', 555),
 ('post amp', 555),
 ('amp cs', 555),
 ('cs apply', 555),
 ('monsoonvalleyuk win bottles', 555),
 ('win bottles fantastic', 555),
 ('bottles fantastic wine', 555),
 ('fantastic wine enter', 555),
 ('wine enter simply', 555),
 ('enter simply follow', 555),
 ('simply follow monsoonvalleyuk', 555)]

In [47]:
from textblob import TextBlob

In [48]:
# Showing tweet sentiment in Bernie df
# Polarity ranges from -1 (most negative) to 1 (most positive).
# Subjectivity ranges from 0.0 (very objective) to 1.0 (very subjective)
for tweet in bernie_df['text']:
    print(tweet)
    analysis = TextBlob(tweet)
    print('---')
    print(analysis.sentiment)
    print('---')

We must not view America only as a population of disconnected individuals.

We must view ourselves as part of “an inescapable network of mutuality, tied in a single garment of destiny,” as Dr. King put it.

That is how we'll realize our vision of justice and dignity for all.
---
Sentiment(polarity=-0.03571428571428571, subjectivity=0.6071428571428571)
---
As president, I will reverse this disastrous rule on day one of my presidency. Together, we will guarantee every person with a disability the right to live in the community with dignity and independence.
---
Sentiment(polarity=-0.09264069264069263, subjectivity=0.611904761904762)
---
10,000 Americans died waiting for Social Security Disability Insurance in 2017. Instead of addressing this crisis, Trump wants to make it harder for people with disabilities to get the help they need to get by.

Unacceptable. We must protect and expand SSDI and SSI. https://t.co/B0XX76AARE
---
Sentiment(polarity=0.044444444444444446, subjectivity=0.055555

---
Our young people did what they were told: they got an education and worked hard.

They're now being punished with crushing student debt and low-paying jobs.

It's about time we stand up for young people and tell the economic elite: you can no longer have it all.
---
Sentiment(polarity=0.10166666666666666, subjectivity=0.4883333333333334)
---
My opponents would tell you that campaign contributions from the wealthy and the powerful don’t have an impact. Why do you think these CEOs are making contributions? https://t.co/fPOGSpp62S
---
Sentiment(polarity=0.4, subjectivity=1.0)
---
I am proud to stand with the students in Rochester who are fighting proposed teacher layoffs. I urge government officials to keep these educators in their classrooms. Together, we will invest in public education and reduce class sizes. https://t.co/YNLMP4lAxv
---
Sentiment(polarity=0.4, subjectivity=0.5333333333333333)
---
You should be able to leave a job or a marriage without fear of losing your health insu

---
Sentiment(polarity=0.5, subjectivity=0.5)
---
This is an emergency. We can not afford half measures.

I am proud to have introduced the only plan that will address the crisis we face. We need a Green New Deal now. https://t.co/nyoW9SwsQv
---
Sentiment(polarity=0.11393939393939397, subjectivity=0.5842424242424242)
---
Thank you, @UNITEHERE, for your courage. @AmericanAir made $1.4 billion in profits last year but refuses to pay workers fairly. When we are in the White House we will take on corporate greed and ensure all workers have a living wage and the right to join a union. https://t.co/3FRXOJGQAg
---
Sentiment(polarity=0.19714285714285712, subjectivity=0.30047619047619045)
---
"He’s not there for Wall Street. He’s not there for Hollywood. He’s not there for big oil, or big pharma or big anybody. Just Big Us. And I’m good with Big Us." https://t.co/lGDGjgBpJH
---
Sentiment(polarity=0.11666666666666665, subjectivity=0.18333333333333335)
---
We are going to live up the values Dr. K

Sentiment(polarity=-0.09722222222222221, subjectivity=0.18055555555555555)
---
We as a nation have a moral responsibility to ensure that all Americans with disabilities have access to the programs and the support needed to contribute to society, live with dignity, and achieve a high quality of life. Join me live in Iowa: https://t.co/n9Z4jGAJ5k
---
Sentiment(polarity=0.10818181818181818, subjectivity=0.4475)
---
I appreciate Gov. Newsom taking this stand. Californians can't remain hostage to a profiteering corporation for a basic need. PG&amp;E's greed and corruption led to gross mismanagement of the state's electric system. Californians deserve a utility that's accountable to the people. https://t.co/b7GgR6rtnl
---
Sentiment(polarity=0.0, subjectivity=0.0625)
---
Last night 1,500 Iowans marched with us to say: enough is enough. The billionaire class cannot have it all. We are going to end the outrageous corporate greed that is devastating working people in America. https://t.co/zn05hy

Sentiment(polarity=0.0, subjectivity=0.06666666666666667)
---
No person needs to have tens of billions of dollars while others sleep out on the streets. We're going to tax the ultra rich and use the revenue to guarantee decent housing for all.
---
Sentiment(polarity=0.2708333333333333, subjectivity=0.7083333333333333)
---
Mental health care shouldn't be a privilege for the wealthy. It should be accessible to all. Medicare for All will guarantee mental health coverage to every person in this country, free at the point of service. #WorldMentalHealthDay
---
Sentiment(polarity=0.21500000000000002, subjectivity=0.515)
---
It's not coincidental that the top health care CEOs made $1.1 billion last year while 4 in 10 cancer patients lose their life savings due to treatment.

Our system is designed to take money from sick people and put it in the pockets of rich executives.

We need Medicare for All.
---
Sentiment(polarity=0.00714285714285714, subjectivity=0.5097619047619047)
---
Let me congrat

Sentiment(polarity=0.0, subjectivity=0.5)
---
Trump promised he would bring down the cost of your prescriptions.

He lied, and drug companies continue to kill Americans with their greed.

I have no intention of letting them continue to get away with murder. We will stand up to Big Pharma and cut drug prices in half.
---
Sentiment(polarity=-0.10740740740740741, subjectivity=0.1851851851851852)
---
The corporate greed of the Sackler family killed thousands and ruined the lives of millions. It’s unacceptable that Purdue Pharma could escape full public accountability through corporate bankruptcy, and its executives who broke the law should go to prison.https://t.co/t9u40Szxcx
---
Sentiment(polarity=0.029999999999999992, subjectivity=0.12333333333333334)
---
The greed of General Motors:

-CEO makes 281x their average worker
-Pays nothing in federal income taxes
-Got a $514 million tax break from Trump

I stand with the 49,000 @UAW GM workers who just went on strike. Together we are going to

Sentiment(polarity=0.1369047619047619, subjectivity=0.3452380952380952)
---
As a nation we must invest in education.

We need to raise teacher pay to at least $60,000 a year.

We need universal pre-K.

We need to cancel student debt.

We need to make public colleges, universities and trade schools tuition-free.
---
Sentiment(polarity=-0.09999999999999999, subjectivity=0.15555555555555556)
---
Tens of thousands of Americans die every year because they lack decent health insurance. There is no middle ground when it comes to heath care. We must guarantee affordable health care to every American through Medicare for All.
---
Sentiment(polarity=0.05555555555555555, subjectivity=0.2222222222222222)
---
Playing it safe, according to the old rules is the most dangerous course we could take. It could cost us the election.

We must reject incrementalism and pursue the transformative change that the working families of this country are crying out for.
---
Sentiment(polarity=0.06000000000000002, s

Sentiment(polarity=-0.06000000000000001, subjectivity=0.31333333333333335)
---
I held an impromptu town hall on the streets of Burlington to hear people's stories about our dysfunctional health care system.

This is why we fight for Medicare for All. https://t.co/w0WNG6IYki
---
Sentiment(polarity=0.0, subjectivity=0.0)
---
The 99 is live from Bernie's rally with UNITE HERE airport workers fighting for a living wage.  American Airlines made $1.9 billion in profit last year — they can afford to pay their workers a living wage. https://t.co/GT3kj6vlCj
---
Sentiment(polarity=0.04545454545454545, subjectivity=0.18888888888888888)
---
It would be a nice idea for Donald Trump to read the Constitution of the United States. https://t.co/1z0It0ejEg
---
Sentiment(polarity=0.6, subjectivity=1.0)
---
Denali Stuckey should be alive today.

Over 140 trans women have been killed since 2013. We must all stand with the trans community against this bigotry, hate and violence. https://t.co/NnKtfL3AtN
---


Sentiment(polarity=0.0, subjectivity=0.0)
---
In 2017 the 25 highest-paid pharma CEOs made $440 million. The top 65 health care CEOs made $1.7 billion.

Our health care system is built to make massive profits for a small few.

Bottom line: no one should profit off illness and suffering. That's why we need #MedicareForAll.
---
Sentiment(polarity=0.012499999999999997, subjectivity=0.5)
---
Yes, we have a proposal to cut prescription drug prices in half. But we also have something else even more important.

We have the guts to take on the pharmaceutical industry and end their greed. https://t.co/VfWmljAbAg
---
Sentiment(polarity=0.24444444444444446, subjectivity=0.5555555555555555)
---
Maybe, just maybe, it's time to start treating this like a crisis and not a hoax. https://t.co/XZKQ8MiqCY
---
Sentiment(polarity=0.0, subjectivity=0.0)
---
No high-priced fundraisers. No super PAC. Our campaign is powered by you and the nearly 2 million donations we’ve received since our launch. Join us tod

Sentiment(polarity=-0.125, subjectivity=0.5)
---
Seven years ago President Obama announced the DACA program. I am inspired by Dreamers and the movement they built for immigration justice. 

This is a promise: As president, I will provide immediate legal status for Dreamers.
---
Sentiment(polarity=0.2, subjectivity=0.2)
---
For over 30 years, janitors across the nation have joined together as part of @SEIU and have won better pay and working conditions. I stand with them and workers everywhere today on National Justice for Janitors Day. https://t.co/PoOgjvGBmg
---
Sentiment(polarity=0.5, subjectivity=0.5)
---
Republicans want to suppress votes and gerrymander districts because they can't win on their ideas. That is cowardice.

We have a better idea: make it easier for all people to vote, run for office and get involved in politics. That is democracy.
---
Sentiment(polarity=0.65, subjectivity=0.45)
---
If you are 18 or older you should be automatically registered to vote.
---
Sentiment(p

In [49]:
# Showing tweet sentiment in wine
for tweet in wine_tweet_df['text']:
    print(tweet)
    analysis = TextBlob(tweet)
    print(analysis.sentiment)

RT @visit_temecula: Come join Chapin Family Vineyards for a glass of wine with a spectacular view! Thanks for sharing this beautiful view 📷…
Sentiment(polarity=0.6, subjectivity=0.7000000000000001)
#winetasting in #manhattanbeach @PasquaWinery #carmenetwine #stefanofarinawine #lucasandlewellen @broadsidewine $10/tasting with discounts on the wines purchased from the flight #wine #familyowned #shopsmall #shoplocal #boxsigns #funnywinequotes https://t.co/B45PGDRHRt
Sentiment(polarity=0.0, subjectivity=0.0)
New Ways Drones Are Changing Vineyard Management https://t.co/xkPBCRZh8g via @SevenFiftyDaily #wine #vineyardmanagement
Sentiment(polarity=0.13636363636363635, subjectivity=0.45454545454545453)
From back in 2016 - 5 #Wine Rules That Really Matter https://t.co/8SVV4kDyPm from @WineSpectator #winelover
Sentiment(polarity=0.1, subjectivity=0.1)
RT @CaraMiaSG: #WiningHourChat returns tonight and we’re kicking off the new year with @WineTravelerCom! Join us at 9PM ET🥂✈️ #wine #travel…
Senti

@MonsoonValleyUK great giveaway #win #wine
Sentiment(polarity=0.8, subjectivity=0.575)
#WiningHourChat returns tonight and we’re kicking off the new year with @WineTravelerCom! Join us at 9PM ET🥂✈️ #wine #travel 

@TheWiningHour @winegal57 @suziday123 @DemiCassiani @LoriMoreno @always5star @Friscokid49 @wineconcubine @LizPalmer_Wine @WineDesTnations @RickGriffin https://t.co/way3jUp4aO
Sentiment(polarity=0.17045454545454544, subjectivity=0.45454545454545453)
RT @MonsoonValleyUK: #Win 3 bottles of fantastic #wine! To enter, simply follow @MonsoonValleyUK and RT this post! T&amp;Cs apply. https://t.co…
Sentiment(polarity=0.43333333333333335, subjectivity=0.5523809523809524)
RT @MonsoonValleyUK: #Win 3 bottles of fantastic #wine! To enter, simply follow @MonsoonValleyUK and RT this post! T&amp;Cs apply. https://t.co…
Sentiment(polarity=0.43333333333333335, subjectivity=0.5523809523809524)
RT @MonsoonValleyUK: The cooling #oceanbreeze and sandy soil of our #vineyard helps to keep our #wine

Sentiment(polarity=0.43333333333333335, subjectivity=0.5523809523809524)
RT @AchemsInfo: Before taking a sip of #wine we perceive its wonderful #aroma. How can we describe the variety of #odors found in a particu…
Sentiment(polarity=1.0, subjectivity=1.0)
RT @vernsanders: another #wine LifeLesson
#bestmedicine https://t.co/CBSmQifyoR
Sentiment(polarity=0.0, subjectivity=0.0)
RT @MonsoonValleyUK: #Win 3 bottles of fantastic #wine! To enter, simply follow @MonsoonValleyUK and RT this post! T&amp;Cs apply. https://t.co…
Sentiment(polarity=0.43333333333333335, subjectivity=0.5523809523809524)
RT @MonsoonValleyUK: #Win 3 bottles of fantastic #wine! To enter, simply follow @MonsoonValleyUK and RT this post! T&amp;Cs apply. https://t.co…
Sentiment(polarity=0.43333333333333335, subjectivity=0.5523809523809524)
The Importance of a Wine’s Acidity https://t.co/viEC0dJBqy via @WineEnthusiast #wine #winelover
Sentiment(polarity=0.0, subjectivity=0.0)
RT @MonsoonValleyUK: #Win 3 bottles of fantastic

Sentiment(polarity=0.0, subjectivity=0.0)
RT @MonsoonValleyUK: #Win 3 bottles of fantastic #wine! To enter, simply follow @MonsoonValleyUK and RT this post! T&amp;Cs apply. https://t.co…
Sentiment(polarity=0.43333333333333335, subjectivity=0.5523809523809524)
Busy Tuesday deserves some #wine &amp; peace for the evening...cheers! (find my #art Thirst for Peace in many sizes: https://t.co/hAdXXfdAwr) #tuesdaythoughts #winetasting #wineart https://t.co/ca1Hgh9kwL
Sentiment(polarity=0.3125, subjectivity=0.4)
RT @MonsoonValleyUK: #Win 3 bottles of fantastic #wine! To enter, simply follow @MonsoonValleyUK and RT this post! T&amp;Cs apply. https://t.co…
Sentiment(polarity=0.43333333333333335, subjectivity=0.5523809523809524)
RT @MonsoonValleyUK: #Win 3 bottles of fantastic #wine! To enter, simply follow @MonsoonValleyUK and RT this post! T&amp;Cs apply. https://t.co…
Sentiment(polarity=0.43333333333333335, subjectivity=0.5523809523809524)
RT @MonsoonValleyUK: #Win 3 bottles of fantastic #wine

Sentiment(polarity=0.11666666666666665, subjectivity=0.3333333333333333)
RT @MonsoonValleyUK: #Win 3 bottles of fantastic #wine! To enter, simply follow @MonsoonValleyUK and RT this post! T&amp;Cs apply. https://t.co…
Sentiment(polarity=0.43333333333333335, subjectivity=0.5523809523809524)
RT @MonsoonValleyUK: #Win 3 bottles of fantastic #wine! To enter, simply follow @MonsoonValleyUK and RT this post! T&amp;Cs apply. https://t.co…
Sentiment(polarity=0.43333333333333335, subjectivity=0.5523809523809524)
#HappyTuesday! #mojitos are only $6 from 6-10pm every Tuesday #HappyHour and #HappyHourSnacks from 4-6pm M-F.  #MojitoMadness #tropical #TropicalTuesday #rum
.
#cocktails #barsnacks #wine #beer #drinks #eveningsout… https://t.co/K52oHQTfYU
Sentiment(polarity=0.0, subjectivity=1.0)
TAKE THE PLUNGE! 🌊🏖✨
Stay up to date on weekly events and deals at the link to our website below!

https://t.co/w3E1pmWSLM 

#HappyHour #Dining #Beer #VeganFood #LongBeach #CaliFoodies #VeganFriendly #PlungeLi

...

Sentiment(polarity=0.7, subjectivity=0.6000000000000001)
Dating for #winelovers explained by emojis MT @CaraMiaSG3 ...#wine ... https://t.co/44M5M80kIj
Sentiment(polarity=0.0, subjectivity=0.0)
RT @MonsoonValleyUK: #Win 3 bottles of fantastic #wine! To enter, simply follow @MonsoonValleyUK and RT this post! T&amp;Cs apply. https://t.co…
Sentiment(polarity=0.43333333333333335, subjectivity=0.5523809523809524)
RT @MonsoonValleyUK: #Win 3 bottles of fantastic #wine! To enter, simply follow @MonsoonValleyUK and RT this post! T&amp;Cs apply. https://t.co…
Sentiment(polarity=0.43333333333333335, subjectivity=0.5523809523809524)
RT @JeannieChoLee: When a wine is this good, it seems to disappear...we all looked at each other after just 30 min &amp; said ‘what happened to…
Sentiment(polarity=0.2875, subjectivity=0.48750000000000004)
OUT TO LUNCH at Cafe Cantina in the North End of Boston @_northendboston #Italian #Meatballs #Raviolis #wine 
@IamMrMonday #CafeCantina

WATCH HERE - https://t.co/o

https://t.co/ikRAqRXkjB
Sentiment(polarity=0.0, subjectivity=0.0)
There is nothing more #satisfying than seeing the #community you serve give back in the form of #support and #trust. #Thankyou to all our previous #customers for #choosingus. Give Us A Call at # (408) 831-7729 today! #Winery #Wine #WineTastings #Soleda ... https://t.co/GQEoQZkFS9 https://t.co/dO9fZPEafK
Sentiment(polarity=0.19791666666666669, subjectivity=0.4166666666666667)
RT @MonsoonValleyUK: #Win 3 bottles of fantastic #wine! To enter, simply follow @MonsoonValleyUK and RT this post! T&amp;Cs apply. https://t.co…
Sentiment(polarity=0.43333333333333335, subjectivity=0.5523809523809524)
Quinta dos Carvalhais Tinta Roriz 2011 Red WineProdutor: Sogrape Vinhos de Portugal, SA
Enólogo: Manuel Vieirahttps://vinsportugal.fr/dao-vins-rouges/2190-quinta-dos-carvalhais-tinta-roriz-2011-red-wine-dao-vins-rouges.html#vins #oporto #portugal #wine #vin #porto #winelover #vis https://t.co/DHnQsywjTJ
Sentiment(polarity=0.0, subjectiv

Sentiment(polarity=0.43333333333333335, subjectivity=0.5523809523809524)
Se terminaron las vacaciones de navidad y ya estamos pensando en la vuelta! ¿Tu no?

Para poder estar en esta maravillosa plaza
.
#restaurante #restaurant #comida #bebida #drink #wine #ronda #serraniaderonda #costadelsol #costadelgolf #marbella https://t.co/Cj4JG8laYm
Sentiment(polarity=0.0, subjectivity=0.0)
💎 Tuesday Deals ➥ Dinner includes a free glass of house wine + 10% off the entree at Mia Regazza (Abington) https://t.co/JFsxVm0XRa 

➲ Check Out all our MA Deals ➟ https://t.co/DPJSy0MKCA 

#Massachusetts #MAdeals #dining #wine #restaurant #southshore Rockland Whitman https://t.co/S8n6Av2R69
Sentiment(polarity=0.4, subjectivity=0.8)
Have you liked our facebook page! Fun #wine questions What would you do scenarios &amp; a whole lot more! https://t.co/gCOgtMqvHx
Sentiment(polarity=0.46875, subjectivity=0.475)
RT @LosaniHomesLtd: @TheTwentyValley knows how to have fun! As Winter WineFest kicks off this Fri. Jan

Sentiment(polarity=0.4, subjectivity=0.4)
RT @RoarLoudTravel: @Julianna_glass @_sundaysunsets_ @always5star Wow, I love the clouds in the glass!!  Gorgeous shot, I love your creativ…
Sentiment(polarity=0.5203125, subjectivity=0.775)
Iconic Brands Issues Shareholder Letter $ICNB #Business #Expansion #Spirits #Wine #Prosecco

Read the full news release here: https://t.co/LglaBNltrd https://t.co/ZRGoW4IIwc
Sentiment(polarity=0.425, subjectivity=0.525)
RT @Julianna_glass: The moment so peaceful, 
so beautiful. 
The moment of quiet thoughts 
and reflections.

The moment of #SundaySunsets wi…
Sentiment(polarity=0.3666666666666667, subjectivity=0.611111111111111)
RT @MonsoonValleyUK: #Win 3 bottles of fantastic #wine! To enter, simply follow @MonsoonValleyUK and RT this post! T&amp;Cs apply. https://t.co…
Sentiment(polarity=0.43333333333333335, subjectivity=0.5523809523809524)
RT @DemiCassiani: Rosé all day ... more like rosé all week long!
.
.
.
#wine @birdinhandwine @LisaRivera2207 @nineov 

Sentiment(polarity=0.43333333333333335, subjectivity=0.5523809523809524)
RT @MonsoonValleyUK: #Win 3 bottles of fantastic #wine! To enter, simply follow @MonsoonValleyUK and RT this post! T&amp;Cs apply. https://t.co…
Sentiment(polarity=0.43333333333333335, subjectivity=0.5523809523809524)
RT @rethink_x: #Beer, #cheese, #wine – fermentation is an age-old #FoodTech process. But, new technologies that use #PrecisionFermentation…
Sentiment(polarity=0.13636363636363635, subjectivity=0.45454545454545453)
RT @DemiCassiani: Rosé all day ... more like rosé all week long!
.
.
.
#wine @birdinhandwine @LisaRivera2207 @nineov @suziday123 @winewanke…
Sentiment(polarity=0.21875, subjectivity=0.45)
RT @KellyMitchell: When I dream of a destination. I dream of Tuscany. https://t.co/FcZ1CIBhAP #TheWIneSiren #wine #vinho #Italy #Tuscany ht…
Sentiment(polarity=0.0, subjectivity=0.0)
Order here https://t.co/tc2nHzQmer
#LondonWineDeliveries #wineuk #italianwine #wine #redwine #winetasting #winelover #winery